# Implementação do ECDSA com a Curva Elíptica prima P-192

## Descrição do Exercício

A ideia do exercício passa por criar toda uma classe em Python que seja capaz de implementar o ***Eliptic Curve Digital Signature Algorithm* (ECDSA)** com o uso de uma das curvas elípticas definidas no FIPS186-4.

Dessa forma, o grupo recorreu ao documento de seu nome **FIS PUB 186-4** que permitiu estabelecer a escolha desta curva. Escolheu-se assim a curva P-192, cujos seus valores *standard* são fornecidos pelo próprio documento em si.

## Descrição da Implementação

Com a curva elíptica prima escolhida, podem-se definir o conjunto de definicões que a classe Python terá e que permitirão no final fazer um pequeno teste em termos de resultados. 

**Com a pesquisa necessária e com a ideia do funcionamento do algortimo em mente, estabelecem-se as seguintes implementacões:**

- **Criação do par das chaves em modo Curva Elíptica:**

  1. *Private Key* - $d_{A}$ - que corresponderá a um inteiro gerado aleatoriamente dentro do intervalo $[1, n-1]$
  2. *Public Key* - $Q_{A}$ - que corresponderá a um ponto da Curva Elíptica tal que $Q_{A} = d_{A} \times G$
      - $G$ é um ponto gerador da Curva Elíptica


- **Criação da assinatura da mensagem em si:**

    1. Cálculo do valor de $e = HASH(m)$, em que $HASH$ corresponderá a uma função de hash criptográfica em que o seu *return* se irá converter em um inteiro
        - A função de $HASH$ será declara à parte
    2. Criação/Escolha de um inteiro $k$ gerado aleatoriamente dentro do intervalo $[1, n-1]$
    3. Cálculo de um ponto da Curva tal que $(x1,y1) = k \times G$ 
        - $G$ continua a ser o mesmo ponto gerador da Curva Elíptica definido inicialmente
    4. Cálculo do valor de $r$ tal que ${\displaystyle r=x_{1}\,{\bmod {\,}}n}$, sendo esse *mod* calculado com base nos ensinamentos de **Corpos Finitos** abordados no **Trabalho 1**.
        - Dessa forma, cria-se um **Corpo Finito** em torno do primo $n$ e aplica-se o valor de $x1$ a esse mesmo tal que $r = F_{n}(x_{1})$
    5. Cálculo do valor de $s$ tal que ${\displaystyle r=x_{1}\,{\bmod {\,}}n}$
        - Assume-se $z$ como sendo o valor calculado para $e$
        - Aplica-se o mesmo princípio do valor de $r$ tal que $s = F_{n}(k)^{-1} \times (e + r \times d)$


- **Verificação dessa mesma assinatura:**

    1. Calcular $e = HASH(m)$, em que $HASH$ terá de ser a mesma função usada para criar a assinatura
    2. Calcular o valor de $w$ tal que ${\displaystyle w=s^{-1}\,{\bmod {\,}}n}$
    3. Calcular o valor de dois valores necessários para um ponto da Curva
        - Calcular o valor de $u_{1}$ tal que ${\displaystyle u_{1}=zw\,{\bmod {\,}}n}$
        - Calcular o valor de $u_{2}$ tal que ${\displaystyle u_{2}=rw\,{\bmod {\,}}n}$
    4. Com os dois valores de cima calcular um ponto da Curva tal que ${\displaystyle (x_{1},y_{1})=u_{1}\times G+u_{2}\times Q_{A}}$
    5. Caso ${\displaystyle r\equiv x_{1}{\pmod {n}}}$, a assinatura é dada como válida
        - Aplicando-se o princípio dos **Corpos Finitos** a assinatura é valida se e só se $r\equiv F_{n}(x_{1})$

Estando todos estes algoritmos definidos e entendidos, desenvolveram-se os metodos necessários para classe Python e através de um mini teste pode-se verificar a verdade de toda esta implementação.

## Resolução do Exercício

In [1]:
import hashlib

class ECDSA():
    
    # Função que inicializa toda a instância e os valores globais necessários à sua execução
    def __init__(self):
        
        # Parâmetros da Curva Elíptica P-192
        # Os parâmetros 'p' e 'n' são dados na forma decimal
        # Os restantes são dados na forma hexadecimal
        self.p = 6277101735386680763835789423207666416083908700390324961279
        self.n = 6277101735386680763835789423176059013767194773182842284081
        self.a = -3
        self.b = 0x64210519E59C80E70FA7E9AB72243049FEB8DEECC146B9B1
        self.Gx = 0x188DA80EB03090F67CBF20EB43A18800F4FF0AFD82FF1012
        self.Gy = 0x07192B95FFC8DA78631011ED6B24CDD573F977A11E794811

        # Corpo Finito Fp em torno do primo p
        self.Fp = FiniteField(self.p)

        # Corpo Finito Fn em torno do primo n
        self.Fn = FiniteField(self.n)

        # Curva Elíptica E
        self.E = EllipticCurve(self.Fp, [self.a, self.b])

        # Ponto Gerador G pertencente à Curva Elíptica E
        self.G = self.E((self.Gx,self.Gy))

    # Função de HASH criptográfica
    # Input: Mensagem para a qual se quer calcular o valor de HASH
    # Output: Devolve o valor de HASH
    def HASH(self, m):
        message = str(m)
        
        return Integer('0x' + hashlib.sha1(message).hexdigest())

    # Algoritmo de criação do par de chaves da Curva Elíptica
    # Input: Não recebe qualquer valor como Input
    # Output: Um par de chaves (Q, d)         
    def keyGenerator(self):
  
        d = randint(1, self.n-1) # Valor Aleatório entre [1, n-1]
        Q = d * self.G
                      
        return (Q, d)
    
    # Algoritmo de criação da assinatura
    # Input: Chave Privada da entidade que quer assinar a mensagem, Mensagem a ser assinada
    # Output: Devolve o par [r,s] a ser usado na verificação da assinatura
    def createSignature(self, d, m):

        e = self.HASH(m)
        
        # Variáveis que vão controlar os dois ciclos
        r = 0
        s = 0
        
        while s == 0:
            # k = 1 ACHO QUE NAO E PRECISO           
            while r == 0:

                # Cálculo do valor de k
                k = randint(1, self.n-1)
                
                # Cálculo do ponto da Curva Elíptica
                P = k * self.G
                (x1, y1) = P.xy()
                
                # Cálculo do valor de r
                r = self.Fn(x1)
            
            # Cálculo do valor de s
            s = self.Fn(k) ^ (-1) * (e + r*d)
        
        return [r,s]
    
    # Algoritmo de verificação da assinatura
    # Input: Chave Pública, Mensagem e par [r,s] obtidos na assinatura da Mensagem Inicial/Original
    # Output: Resultado da verificação
    def verifySignature(self, Q, m, r, s):
        
        e = self.HASH(m)
        
        w = s ^ (-1)
        
        u1 = (e * w)
        u2 = (r * w)
        
        P1 = Integer(u1) * self.G
        P2 = Integer(u2) * Q
        P = P1 + P2
        (x1, y1) = P.xy()
        
        return r == self.Fn(x1)
                

## Teste da Classe

Segue-se abaixo um teste desenvolvido para esta classe. **A ideia passa pelos seguintes passos:**

- Gerar um par de chaves (Q,d)
- Criar uma mensagem inicial **"A mensagem que vou pedir para assinar"**
- Assinar essa mensagem inicial com a *Private Key* $d_{A}$
- Criar uma nova mensagem **"Vou tentar uma nova mensagem"**
- Verificar a assinatura com essa nova mensagem

Isto vai permitir verificar que o resultado é ***false*** dado que não corresponde à mensagem inicialmente assinada pela chave em si.

In [2]:
# Criar uma instância e sua inicialização
myECDSA = ECDSA()

# Obter o par de chaves da Curva Elíptica
(Q, d) = myECDSA.keyGenerator()

print "Eliptic Curve Public Key - ", Q.xy()
print "Eliptic Curve Private Key - ", d

# Assinar a mensagem
originalMessage = 'A mensagem que vou pedir para assinar'
[r, s] = myECDSA.createSignature(d, originalMessage)

print "Mensagem Original - ", originalMessage

# Verificar a assinatura da mensagem
fakeMessage = 'Vou tentar uma nova mensagem'
result = myECDSA.verifySignature(Q, fakeMessage, r, s)

print "Resultado - ", result

Eliptic Curve Public Key -  (2038752860592359119426082346061557950338888622821805043945, 306303741022623073521196141447092461082295559180875188991)
Eliptic Curve Private Key -  2009916947488637617038617525489375040057193637677791988684
Mensagem Original -  A mensagem que vou pedir para assinar
Resultado -  False


## Observações Finais

- O algoritmo **ECDSA** encontra-se detalhadamente descrito em várias fontes *online*
- Depois de compreender o seu funcionamento torna-se mais simples de começar a aplicar toda a ideia em SageMath

## Referências

- Cameron F. Kerry, Acting Secretary,Patrick D. Gallagher. (julho 2013). FEDERAL INFORMATION PROCESSING STANDARDS
PUBLICATION, Digital Signature Standard (DSS)
 https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-4.pdf
- Wikipedia, Elliptic Curve Digital Signature Algorithm https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm (Acedido a 13 abril 2020)